Copyright D. Bazzett 2020  
This is based off the github repo "earth-cam-downloader" by others. The purpose of this code is to download archived data from the Earth Cam website. The Earth Cam website has no apparent download options for live or archived video. Archived videos are kept in 1 hour-long segments for 2-3 days after they were taken, at which point they are removed.

In addition to the python packages below, you also need ffmpeg, which can be installed by following the instructions in this video. I had to restart after installing to get it to work.  
https://youtu.be/qjtmgCb8NcE


In [2]:
# !pip install ffmpy
# !pip install m3u8
# !pip install earth_cam_downloader

In [3]:
# This is code by others that I did not modify.
# Start

"""
M3U8 Downloader
Download the ts files according to the given m3u8 file.
"""
import argparse
import os

import ffmpy
import m3u8
# from ffmpy import FFmpeg as ffmpeg

class M3U8Downloader:
    """M3U8 Downloader Class"""

    def __init__(
        self,
        uri,
        timeout=None,
        headers=None,
        select_first_playlist=False,
        ffmpeg_path="ffmpeg",
        ffmpeg_loglevel="quiet",
    ):
        """Initialize a M3U8 Downloader.
        Args:
            uri (:obj:`str`): The URI of the m3u8 file.
            timeout (:obj:`int`, optional): The timeout used when loading
                from uri. Defaults to ``None``.
            headers (:obj:`list` of :obj:`str`, optional): The headers used
                when loading from uri. Defaults to ``None``.
            ffmpeg_path (:obj:`str`, optional): The path to ffmpeg executable.
                Defaults to ``ffmpeg``.
            ffmpeg_loglevel (:obj:`str`, optional): The logging level of
                ffmpeg. Defaults to ``quiet``.
            select_first_playlist(:obj:`boolean`, optional): Whether or not to simply select
                the first playlist for a variant m3u8 file.
        """
        if not headers:
            headers = {}

        self.uri = uri
        self.ffmpeg_path = ffmpeg_path
        self.ffmpeg_loglevel = ffmpeg_loglevel
        self.select_first_playlist = select_first_playlist
        self.m3u8 = m3u8.load(uri=uri, timeout=timeout, headers=headers)

    def download(self, output="output.ts"):
        """Start downloading and merging with the given m3u8 file.
        Args:
            output (:obj:`str`): The path to output. Defaults to ``output.ts``
        """
        if self.m3u8.is_variant:
            print("INFO: There are multiple m3u8 files listed in this file.")
            try:
                if self.select_first_playlist:
                    print("INFO: Simply selecting the first playlist to download...")
                    fetch_index = 0
                else:
                    print()
                    for index, playlist in enumerate(self.m3u8.playlists):
                        self._print_stream_info(playlist, index)
                    fetch_index = int(input("Index> "))

                downloader = M3U8Downloader(
                    self.m3u8.playlists[fetch_index].absolute_uri,
                    ffmpeg_path=self.ffmpeg_path,
                    ffmpeg_loglevel=self.ffmpeg_loglevel,
                )
                downloader.download(output)
            except (ValueError, IndexError):
                print("ERROR: Invalid index.")

        else:
            dirname = os.path.dirname(output)
            if dirname:
                os.makedirs(os.path.dirname(output), exist_ok=True)
            ffmpeg_cmd = ffmpy.FFmpeg(
                self.ffmpeg_path,
                "-y -loglevel {}".format(self.ffmpeg_loglevel),
                inputs={self.uri: None},
                outputs={output: "-c copy"},
            )
            print("INFO: Start downloading and merging with ffmpeg...")
            print(ffmpeg_cmd.cmd)
            ffmpeg_cmd.run()

    @staticmethod
    def _print_stream_info(playlist, index=0):
        print("INDEX: " + str(index))

        stream_info = playlist.stream_info
        if stream_info.bandwidth:
            print("Bandwidth: {}".format(stream_info.bandwidth))
        if stream_info.average_bandwidth:
            print("Average bandwidth: {}".format(stream_info.average_bandwidth))
        if stream_info.program_id:
            print("Program ID: {}".format(stream_info.program_id))
        if stream_info.resolution:
            print("Resolution: {}".format(stream_info.resolution))
        if stream_info.codecs:
            print("Codecs: {}".format(stream_info.codecs))
        print()

# End

In [6]:
# https://www.earthcam.com/search/ft_search.php?term=seaside+heights

# !/usr/bin/env python3

import time
from datetime import datetime

from earth_cam_downloader import EarthCamDownloader

# note: the start and end times must be changed to your desired window
# only available for 2-3 days after initial capture - act fast!
# start and end times will download videos in 1 hour segments
# ex 1530 to 1630 will download 1500-1600 and 1600-1700

### USER INPUTS ###

# time to start downloading archived footage from
START = datetime(2020, 8, 4, 6, 0, 1) # yyyy m d h m s 

# time to end downloading archived footage from
END = datetime(2020, 8, 4, 12, 0, 0)

# camera names and ids to download
CAMERAS = {
#     "seasideheights":  "5173", # North
    "seasideheights2": "6975", # South, doesn't work for some reason?
#     "seasideheights3": "15276", # Amusement Park
#     "seasidepark": "15305",
    
#     "times-square-4k": "hdtimes10",
#     "crossroads": "15559",
#     "south-view": "4017timessquareHD1",
}

FILE_FORMAT = "mp4"

OUTPUT_DIR = "output"

### END USER INPUTS ###

def main():
    started_at = time.time()
    for cam_name, cam_id in CAMERAS.items():
        EarthCamDownloader(
            cam_id, cam_name, START, END, FILE_FORMAT, OUTPUT_DIR
        ).download()
    ended_at = time.time()
    print(f"Downloaded video files in {round((ended_at -  started_at) / 60, 2)} minutes.")


if __name__ == "__main__":
    main()

    #https://video2archives.earthcam.com/archives/_definst_/MP4:network/6975/2020/08/04/0601.mp4/playlist.m3u8
#    https://video2archives.earthcam.com/archives/_definst_/MP4:network/6975/2020/08/04/0600.mp4/playlist.m3u8 

INFO: Downloading https://video2archives.earthcam.com/archives/_definst_/MP4:network/6975/2020/08/04/0600.mp4/playlist.m3u8 to output/seasideheights2-2020-08-04-0600.mp4


HTTPError: HTTP Error 404: Not Found